In [10]:
import pandas as pd
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
ratio_dist=25

In [2]:
input_locations=pd.ExcelFile("./Events1008.xlsx")
input_locations.sheet_names

['sampleevents-3', 'Sheet1']

In [3]:
input_locations=input_locations.parse("Sheet1",dtype=str)
input_locations.head(2)

,ancestors,createdDate,currencyCode,description,eventDateLocal,eventDateUTC,hideEventDate,hideEventTime,id,lastUpdatedDate,...,venue_city,venue_state,venue_postalCode,venue_country,venue_venueConfigId,venue_venueConfigName,ticketInfo_minListPrice,ticketInfo_maxListPrice,ticketInfo_totalTickets,ticketInfo_totalListings
0,"{'categories': [{'id': 1, 'name': 'Concert'}],...",2019-06-07T15:49:38+0000,USD,Billy Joel Madison Square Garden Tickets - Buy...,2019-12-11T20:00:00-0500,2019-12-12T01:00:00+0000,False,False,104180213,2019-07-29T13:50:23+0000,...,New York,NY,10001,US,588385,End Flr A-F - Dynamic,71,9999,4654,843
1,"{'categories': [{'id': 1, 'name': 'Concert'}],...",2019-05-11T15:43:04+0000,USD,Jonas Brothers The Forum Los Angeles Tickets -...,2019-12-14T19:30:00-0800,2019-12-15T03:30:00+0000,False,False,104146867,2019-07-11T21:47:11+0000,...,Inglewood,CA,90305,US,1226841,Jonas Brothers 2019 - Dynamic,57,10548.95,3511,1103


In [4]:
df_TA=input_locations[['venue_id','venue_name','venue_city','venue_state','venue_postalCode','venue_country']].drop_duplicates()
df_TA.shape,df_TA['venue_id'].nunique()

((4, 6), 4)

In [5]:
df_TA['search_key']=df_TA.apply(lambda x: ", ".join(list([x['venue_name'],x['venue_city'],x['venue_state'],x['venue_postalCode'],x['venue_country']])),axis=1)


In [6]:
location_input_dict=df_TA.set_index("venue_id").to_dict()['search_key']
location_input_dict

{'102': ' Oakland Arena,  Oakland,  CA, 94621,  US',
 '1282': ' Madison Square Garden,  New York,  NY, 10001,  US',
 '5905': ' Mohegan Sun Arena,  Uncasville,  CT, 6382,  US',
 '753': ' The Forum Los Angeles,  Inglewood,  CA, 90305,  US'}

In [7]:
# Manully copied here through browser

dict_store_locations={}

# '102': ' Oakland Arena,  Oakland,  CA, 94621,  US'
dict_store_locations.update({"102":{"Google_Address":"7000 Coliseum Way, Oakland, CA 94621","Google_Coor":[37.7516081,-122.2007013]}})
# '1282': ' Madison Square Garden,  New York,  NY, 10001,  US'
dict_store_locations.update({"1282":{"Google_Address":"4 Pennsylvania Plaza, New York, NY 10001","Google_Coor":[40.7504956,-73.9935825]}})
# '5905': ' Mohegan Sun Arena,  Uncasville,  CT, 6382,  US'
dict_store_locations.update({"5905":{"Google_Address":"1 Mohegan Sun Blvd, Uncasville, CT 06382","Google_Coor":[41.491589,-72.0916619]}})
# '753': ' The Forum Los Angeles,  Inglewood,  CA, 90305,  US'
dict_store_locations.update({"753":{"Google_Address":"3900 W Manchester Blvd, Inglewood, CA 90305","Google_Coor":[33.9582053,-118.3420794]}})


In [8]:
df_TA['Google_Address']=df_TA['venue_id'].apply(lambda x: dict_store_locations[x]['Google_Address'])
df_TA['Google_Coordinate']=df_TA['venue_id'].apply(lambda x: dict_store_locations[x]['Google_Coor'])

In [13]:
df_TA=df_TA.reset_index()
del df_TA['index']

df_store_zip=pd.DataFrame()

for ind,row in df_TA.iterrows():
    store_center=row['Google_Coordinate']
    zip_list=[]
    store_zip=pd.DataFrame()
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,miles=True) # unit="mi"
        if dist<=ratio_dist:
            zip_list.append(zip_cd)
            df=pd.DataFrame({"dist_miles":dist},index=[zip_cd])
            store_zip=store_zip.append(df)
    store_zip=store_zip.reset_index().rename(columns={"index":"zip_cd"})
    store_zip['venue_id']=row['venue_id']
    df_TA.loc[ind,'zip_list']=str(zip_list)
    df_store_zip=df_store_zip.append(store_zip)

In [15]:
df_TA['zip_list']=df_TA['zip_list'].apply(eval)

In [17]:
set(df_TA.loc[1,'zip_list']).intersection(set(df_TA.loc[2,'zip_list'])) # No obvious overlap

set()

In [19]:
temp_df=input_locations[['venue_id','venue_name','venue_city','venue_state','venue_postalCode','venue_country']].drop_duplicates()
df_store_zip=pd.merge(temp_df,df_store_zip,on="venue_id",how="left")
df_store_zip.shape

(1391, 8)

In [21]:
df_store_zip['zip_cd'].nunique()

1391

In [22]:
writer=pd.ExcelWriter("./Trade_area_for_4_venue_locations_25_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_zip.to_excel(writer,"by_store_zip",index=False)
df_TA.to_excel(writer,"4_locations",index=False)
writer.save()